## Import data

In [111]:
dataCols = [
    'Design with users',
    'Iterate and improve frequently',
    'Work in the open by default',
    'Use open standards and solutions',
    'Address security and privacy risks',
    'Build in accessibility from the start',
    'Empower staff to deliver better services',
    'Be good data stewards',
    'Design ethical services',
    'Collaborate widely'
]

numDataCols = int(len(dataCols))

import csv
import numpy as np

def findStrElement(element, array):
    for item in array:
        if str(item) == str(element):
            return item

def quarterObjectExists(quarter, quarterArray):        
    for lQuarter in quarterArray:
        if str(quarter) == str(lQuarter):
            return True
    return False

In [112]:
quarters = {}
counter = 0;
    
with open('digitalStandardsTabSeparatedData.txt', newline = '') as data:
    data_reader = csv.reader(data, delimiter='\t')
    
    for data in data_reader:
        counter+=1
        if counter == 1: ## Skip the header
            continue
        
        quarterStr = data[0] + "-Q" + data[1]
        data_arr = [
            int(data[2]), int(data[3]), int(data[4]), 
            int(data[5]), int(data[6]), int(data[7]), 
            int(data[8]), int(data[9]), int(data[10]), 
            int(data[11])
        ]
        
        if quarterStr in quarters:
            quarters[quarterStr] = np.append(quarters[quarterStr], data_arr)
        else:
            quarters[quarterStr] = np.array(data_arr)

## Chart Overall Alignment

In [113]:
import plotly.graph_objects as go
import datetime

def getSumDigitalStandardsArr(npDictionary):
    totalsArr = []
    
    for key in npDictionary:
        total = sum(quarters[key])
        numOfSubmissions = (len(quarters[key])/numDataCols)
        totalsArr.append(total/numOfSubmissions)
    
    return totalsArr
    
def getQuartersArr(quarters):
    quartersArr = []
    for quarter in quarters:
        quartersArr.append(str(quarter))
    return quartersArr

In [114]:
x = getQuartersArr(quarters)
y = getSumDigitalStandardsArr(quarters)

fig = go.Figure(data=[go.Scatter(x=x, y=y)])

fig.update_layout(xaxis_range=quartersArr)
fig.show()

In [115]:
def reshapeDsArray(quarters):
    quartersArr = {}
    for key in quarters:
        int_quarters_arr_len = int(len(quarters[key]))
        quartersArr[key] = np.reshape(quarters[key], (int(int_quarters_arr_len/numDataCols), numDataCols))
    
    return quartersArr

def sumDigitalStandardByQuarter(reshapedDsArray):
    summedDsByQuarterArr = []
    for key in reshapedDsArray:
        reshapedArr = reshapedDsArray[key].sum(axis=0)
        reshapedArr = np.reshape(reshapedArr, (numDataCols, 1))
        summedDsByQuarterArr.append(reshapedArr)
        
    summedDsByQuarterArr = np.concatenate((summedDsByQuarterArr), axis=1)
        
    return summedDsByQuarterArr       

In [116]:
x=getQuartersArr(quarters)
ys=sumDigitalStandardByQuarter(reshapeDsArray(quarters))

counter = 0
for y in ys:
    if counter == 0:
        fig = go.Figure(go.Bar(x=x, y=y.tolist(), name=dataCols[counter]))
    else:
        fig.add_trace(go.Bar(x=x, y=y.tolist(), name=dataCols[counter]))
    counter+=1
    
fig.update_layout(barmode='stack')
fig.update_xaxes(categoryorder='category ascending')
fig.show()    

In [117]:
x=getQuartersArr(quarters)
summedDigitalStandardsByQuarter = sumDigitalStandardByQuarter(reshapeDsArray(quarters))

counter = 0
for y in summedDigitalStandardsByQuarter:
    fig = go.Figure([go.Bar(x=x, y=y.tolist())])
    fig.update_layout(
    title={
        'text': dataCols[counter]
    })
    fig.show()
    counter+=1